In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import json
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [44]:
from langchain_core.tools import tool
from langchain_deepseek import ChatDeepSeek
from typing_extensions import Annotated, TypedDict, Literal
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, InjectedState
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage

In [5]:
from src.utils_stock import get_top_nasdaq_stock_data

In [6]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [7]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    number_of_days_analysis: int
    email_address: str

### Data Acquisition Agent

In [8]:
@tool
def get_top_nasdaq_performance_stock_data(
    state: Annotated[State, InjectedState]
) -> str:
    """

    Get the day's top NASDAQ-100 gainer with symbol, percentage increase, sample data in markdown format, pandas dataframe info and csv data path for further python code analysis.

    Returns:
        str: Top nasdaq performance stock data information
    """

    result = get_top_nasdaq_stock_data(state['number_of_days_analysis'])
    return json.dumps(result)

In [9]:
data_acquisition_tools = [get_top_nasdaq_performance_stock_data]

In [10]:
data_acquisition_tools_node = ToolNode(data_acquisition_tools)

In [11]:
data_acquisition_agent = llm.bind_tools(data_acquisition_tools)

In [13]:
def data_acquisition_agent_node(state: State):
    messages = state["messages"]
    response = data_acquisition_agent.invoke(messages)
    return {"messages": [response]}

In [41]:
# Define the conditional edge between the nodes
def data_acquisition_reflection(state: State) -> Literal["data_acquisition_tool", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "data_acquisition_tool"
    return "__end__"

In [42]:
workflow = StateGraph(State)
workflow.add_node("data_acquisition_agent", data_acquisition_agent_node)
workflow.add_node("data_acquisition_tool", data_acquisition_tools_node)

workflow.add_edge("__start__", "data_acquisition_agent")
workflow.add_conditional_edges(
    "data_acquisition_agent",
    data_acquisition_reflection,
)
workflow.add_edge("data_acquisition_tool", "data_acquisition_agent")

graph = workflow.compile()

In [43]:
graph.get_graph().print_ascii()

              +-----------+                     
              | __start__ |                     
              +-----------+                     
                    *                           
                    *                           
                    *                           
       +------------------------+               
       | data_acquisition_agent |               
       +------------------------+               
            ...           ***                   
          ..                 **                 
        ..                     **               
+---------+           +-----------------------+ 
| __end__ |           | data_acquisition_tool | 
+---------+           +-----------------------+ 


In [45]:
input_message = HumanMessage(
    content="First, Let get the top performance stock data")

In [46]:
events = graph.stream(
    input={
        "messages": [input_message],
        "number_of_days_analysis": 10,
        "email_address": "nelsonlin0321@gmail.com"
    },
    # Maximum number of steps to take in the graph
    config={"recursion_limit": 15},
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

First, Let get the top performance stock data
================================== Ai Message ==================================
Tool Calls:
  get_top_nasdaq_performance_stock_data (call_0_e78e8da9-6532-409e-a58f-e3f7860b7d82)
 Call ID: call_0_e78e8da9-6532-409e-a58f-e3f7860b7d82
  Args:


100%|██████████| 101/101 [00:02<00:00, 37.41it/s]


================================= Tool Message =================================
Name: get_top_nasdaq_performance_stock_data

{"symbol": "MNST", "percentage_increased": 3.7238540649033007, "sample_data_in_markdown": "| Date                      |   Open |   High |   Low |   Close |   Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|------:|--------:|---------:|------------:|---------------:|\n| 2025-04-28 00:00:00-04:00 |  58.74 |  58.98 | 58.03 |   58.49 |  3057100 |           0 |              0 |\n| 2025-04-29 00:00:00-04:00 |  58.59 |  59.33 | 58.01 |   59.25 |  3448100 |           0 |              0 |\n| 2025-04-30 00:00:00-04:00 |  59.59 |  60.29 | 58.8  |   60.12 |  5228700 |           0 |              0 |\n| 2025-05-01 00:00:00-04:00 |  59.86 |  60.03 | 59.3  |   59.52 |  5164900 |           0 |              0 |\n| 2025-05-02 00:00:00-04:00 |  59.87 |  60.26 | 59.43 |   60.05 |  4161600 |           0 |              0 |", "data_csv_path": "/